# Caso: Predicción de Tiempo de Espera en Caja

## Objetivo
Predecir el tiempo de espera a ser atendido en caja en una sucursal bancaria.

In [ ]:
# Librerias a importar
import math as math
import numpy as np
import pandas as pd
import pydotplus
import seaborn as sns
import copy
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

%matplotlib inline

from IPython.display import Image

from sklearn.tree import export_graphviz

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import make_scorer
from sklearn import metrics

from sklearn.svm import SVR
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

from sklearn import datasets, linear_model
from sklearn.preprocessing import LabelEncoder


## 1. Análisis descriptivo del dataset

El set de datos a utilizar es **espera_caja.csv** el cual contiene información sobre atenciónes en caja de una sucursal bancaria.

La variable target es **ESPERA_SEQ1**, que especifica el tiempo de espera que tuvo la persona para ser atendida.

In [ ]:
# Leemos el dataset
dataset_entrenamiento='https://raw.githubusercontent.com/unlam-fcdin/UNLaM_FCDIN/master/espera_caja.csv'
df = pd.read_csv(dataset_entrenamiento, sep =',', na_values = '.')

# Para estandarizar el análisis, llamaremos variable "CLASE" a la variable target
df['CLASE'] = df.apply(lambda x: None if x['ESPERA_SEQ1'] is None or math.isnan(x['ESPERA_SEQ1']) else int(x['ESPERA_SEQ1']), axis=1)
df.drop(["ESPERA_SEQ1"], axis=1, inplace=True)

print(df.shape)
df.head(5)

#### **Descripción Estadística General**
Veamos qué información relevante obtenemos de este primer análisis

In [ ]:
# ¿Qué información estadística obtenemos del dataset?
df.describe()

In [ ]:
# Veamos la distribución de la variable target
print(round(df['CLASE'].mean()))     # Media
print(round(df['CLASE'].median()))   # Mediana
print(round(df['CLASE'].std()))      # Desvío

plt.figure(figsize=(20,10))
sns.distplot((df)['CLASE'], hist=True, bins=100, color = 'darkblue',
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})

### 1.a) Análisis de Missing Values o Nulos

Analizamos la existencia de valores faltantes y seleccionamos la estrategia de imputación.

In [ ]:
val_nulos = df.isnull().sum()
print(df.shape)
print(val_nulos)

In [ ]:
# Borramos los casos donde la clase es null, ya que no sirven para entrenamiento
print(df.shape)
df = df[~df.CLASE.isnull()]
print(df.shape)

In [ ]:
# Graficamos la distribución de valores nulos en cada variable
atributos_nulos = (df.isnull()).sum(axis=0)
atributos_nulos = pd.DataFrame(atributos_nulos, columns=['Cantidad de Nulos'])
atributos_nulos = atributos_nulos.sort_values(by=['Cantidad de Nulos'], ascending=True)
atributos_nulos.drop(['CLASE'], inplace = True)
atributos_nulos.plot(kind='barh', figsize=(15,5), color='orange', grid=False)

### 1.b) Análisis de Outliers

Analizamos los valores outliers de cada variable y determinamos si queremos/debemos imputarlos o no.

In [ ]:
# Creamos un gráfico boxplot para analizar las variables
for c in df.columns:
  if("float" in str(df.dtypes[c]) or "int" in str(df.dtypes[c])):
    plt.figure(figsize=(10, 3))
    sns.boxplot(x=c, data=df, orient = 'h', palette="Set2")

## 2. Preparación de datos

Una vez cargados los datos y analizados, se deben preparar para ser procesados. Los 3 pasos generales que se deben seguir son:

1. Tratamiento de outliers
2. Feature engineering
3. Tratamiento de missing values (o valores nulos)

Los pasos detallados de la fase, son:

- Leer los datos con Pandas.
- Comprobar si hay valores nulos y crear todas las variables nuevas.
- Construir una variable Y que contenga la variable objetivo a predecir y una vector X que contenga todo el resto de variables a usar en la predicción.
- Dividir el dataset completo en training y testing
- Dividir X e y con train_test_split así:
        train_test_split(X, y, test_size=0.3, random_state=42)

Vamos a crear una función llamada **preparacion_de_datos(dataset)** que incluirá todas las tareas de preparación de datos necesarias para construir nuestro modelo predictivo. Esta función la vamos a utilizar tanto para el dataset de entrenamiento como para el de prueba.


In [ ]:
from sklearn.preprocessing import StandardScaler

# FUNCION preparacion_de_datos (:parametros)
#   df_e            => Dataset de entrada a modificar
#   imputar_ouliers => Flag que indica si se deben imputar outliers o no
#   imputar_nulos   => Flag que indica si se deben imputar nulos o no

def preparacion_de_datos(df_e, imputar_ouliers, imputar_nulos):

  # Comenzamos haciendo una copia del dataset que la función recibe como parámetro de entrada
  df_s = copy.copy(df_e)

  # (*1) ---- IMPUTACIÓN DE OUTLIERS ----

  # En este punto se deben imputar los valores outliers, sólo si se indicó por parámetro(imputar_outliers)
  if imputar_ouliers:
    print("TODO: Imputación de outliers.")

    # Primero definimos una función para calcular la media, con los parámetros:
    ## dff -> Dataframe a usar
    ## c   -> Columna a imputar
    ## min -> Límite inferior, si no aplica no se envía.
    ## max -> Límite superior, si no aplica no se envía.

    def calcular_media(dff, c, min=None, max=None):
      # Seteamos el mínimo y máximo, si viene por parámetro. Si no fueron informados, tomamos el actual del set de datos.
      minimo = dff[c].min() if min==None else min
      maximo = dff[c].max() if max==None else max

      # Filtramos los registros dentro del rango
      dff2 = dff[(dff[c]>=minimo) & (dff[c]<=maximo)]

      # Devolvemos la media
      return dff2[c].mean()

    # Imputamos por la media los outliers superiores
    outlier_superior = df_s['VISITAS_CAJA'].mean() + 1.5*df_s['VISITAS_CAJA'].std()
    media_sin_outliers = calcular_media(df_s, 'VISITAS_CAJA', max=outlier_superior)
    df_s['VISITAS_CAJA'] = df_s.apply(lambda x: media_sin_outliers if x['VISITAS_CAJA']>outlier_superior else x['VISITAS_CAJA'], axis=1)

    # Imputamos por la media los outliers superiores
    outlier_superior = df_s['FRECUENCIA_ENCUESTAS'].mean() + 2*df_s['FRECUENCIA_ENCUESTAS'].std()
    media_sin_outliers = calcular_media(df_s, 'FRECUENCIA_ENCUESTAS', max=outlier_superior)
    df_s['FRECUENCIA_ENCUESTAS'] = df_s.apply(lambda x: media_sin_outliers if x['FRECUENCIA_ENCUESTAS']>outlier_superior else x['FRECUENCIA_ENCUESTAS'], axis=1)

  # (*2) ---- FEATURE ENGINEERING ----

  # Cada uno puede crear los atributos que considere necesario (y mejoren la predicción)
  print("TODO: Creación de nuevas variables.")

  # (*3) ---- TRATAMIENTO DE VALORES NULOS ----

  # Veamos si tenemos valores nulos o infinitos. Como ejemplo, podemos optar por setear el valor 0 por defecto.
  if imputar_nulos:
    print("TODO: Imputación de valores nulos.")

    # Al resto los imputo por la media
    df_s[df_s==np.inf]=np.nan
    for c in df_s.columns:
      if(not("str"  in str(df_s.dtypes[c]) or "object" in str(df_s.dtypes[c]))):
        df_s[c].fillna(df_s[c].mean(), inplace=True)

  return df_s

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Ejecutamos la función de preparacion de datos.
# Indicando que si queremos imputar ouliers y si queremos imputar nulos
# Este el primer paso que debemos realizar para tener todas las variables a utilizar.
df = preparacion_de_datos(df, True, True)
X = pd.get_dummies(df.drop(["CLASE"],axis = 1),  dtype=int)

# Escalamos algunos de los atributos para que estén entre 0 y 1
atributos = X.columns
#num_vars = ['Desc_Resp_Cerrada','EDAD','FRECUENCIA_ENCUESTAS','VISITAS_CAJA']
#X[atributos] = scaler.fit_transform(X[atributos])

y = df['CLASE']

X.head()

In [ ]:
# Dividimos el dataset en entrenamiento y prueba (70% para training y 30% para testing)
# Dividimos X e y con la funcion train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.3,
                                                    random_state=42)

print(X_train.shape)
X_train.head(5)

#### **Preparación de Datos**
**Consignas**
- ¿Se observa algo objetable en el set de datos generado?

- ¿Hay alguna variable que requiera un tratamiento adicional?


Cabe resaltar que cualquier actividad a realizar sobre el set de datos, debe realizarse en la fase de **Preparación de Datos**.

### Análisis de Correlación de Variables Numéricas

Una matriz de correlación permite estudiar la relación lineal o comportamiento que puede existir entre dos o más variables.

  - Correlación positiva: ocurre cuando una variable aumenta y la otra también.
  - Correlación negativa: es cuando una variable aumenta y la otra disminuye.
  - Sin correlación: no hay una relación aparente entre las variables.

In [ ]:
# Seleccionamos sólo la correlación de la variable objetivo numérica CLASE.
X_trainw = copy.copy(X_train)
X_trainw["CLASE"] = y_train
dfd = X_trainw.corr()[["CLASE"]]*100
del X_trainw

# Borramos la correlación de la variable objetivo numérica consigo misma.
dfd = dfd.drop("CLASE", axis=0)

# Ordenamos las variables de forma decreciente por el valor de correlación positiva con la variable objetivo.
dfd = dfd.sort_values(["CLASE"], ascending=False)

dfd

## 3. Construcción del modelo predictivo de regresión lineal sin optimizacion de hiperparámetros
### 3.a) Modelo inicial

Vamos a construir un modelo de regresión lineal inicial usando el datasetde entrenamiento. Los pasos que realizaremos son:

- Ajustar un modelo regresión lineal con parámetros `default`.
- Calcular la importancia de los atributos.
- Calcular y mostrar el error cometido por el modelo.
- Quitar variables con baja importancia y ver si la regresión mejora.

In [ ]:
# Ajustamos una Regresión Lineal con los parametros default
lr = LinearRegression()
lr.fit(X_train, y_train)

### 3.b) Interpretación de resultados
- La interpretación de la regresión, se realiza en los coeficientes de las variables utilizadas.
- Estos coeficientes dan una idea de la importancia de cada variable para el modelo generado.

In [ ]:
pd.DataFrame({'Atributo':atributos,
              'importancia':(lr.coef_)}).sort_values('importancia',
                                                        ascending=False)

El método antiguo

In [ ]:
import statsmodels.api as sm
from scipy import stats

X2 = sm.add_constant(X_train)
est = sm.OLS(y_train, X2)
est2 = est.fit()

print(est2.summary())

### 3.c) Evaluación de modelos
En el caso de Regresiones, para evaluar qué modelo es mejor/peor, vamos a evaluar la distancia del valor predicho sobre el valor real. En este caso cuantos segundos por exceso, o por defecto, predecimos vs los segundos reales de espera.

Con el modelo ya entrenado, utilizamos el conjunto de prueba para verificar su capacidad de predicción. Existen diversas métricas para esta medición, pero una de las más utilizadas es la métrica de Error Cuadrático Medio (**MSE**) o su versión ajustada **RMSE** (Raíz del Error Cuadrático Medio), cuyas fórmulas son:

$$MSE = {\frac{\sum_{t=1}^n (y_t-\hat y_t )^2}{n}}$$  
$$RMSE = \sqrt{\frac{\sum_{t=1}^n (y_t-\hat y_t )^2}{n}}$$

In [ ]:
# Realizamos la predicción sobre el set de testing
y_pred = lr.predict(X_test)

# MSE - Error cuadratico medio
print('Mean squared error: %.4f'% mean_squared_error(y_test, y_pred))

In [ ]:
# Intentamos calcular a mano el error cuadratico medio para ver que coincide
N   = X_test.shape[0]
MSE = (1/N) * (sum([math.pow(diff,2) for diff in y_test - y_pred]))

print('MSE MANUAL: %.4f'% MSE)
print('RMSE MANUAL: %.4f'% math.sqrt(MSE))

#### Distribución de los errores

Como una conclusión de este análisis, es que nuestro modelo falla en un rango de **13.5 segundos** en promedio dentro de todo el set de datos.

En algunos casos este fallo es mayor y en otros menos. Veamos la distribución de los errores:

In [ ]:
errores = [diff for diff in y_test - (y_pred-4)]

plt.figure(figsize=(20,10))
sns.distplot(errores, hist=True, kde=True,
             bins=50, color = 'darkblue',
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})

#### Coeficiente de determinación R2
Por otro lado, para entender que poder predictivo tiene el modelo podemos analizar el coeficiente de determinación o R2, el cual nos indica que % de la varianza total de la variable objetivo, es explicada por el modelo generado.

El valor 1 simboliza que el modelo explica el 100% de la varianza del set de datos, lo cual indicaría que es un modelo sobre ajustado, valores muy bajos indican que es un modelo sub-ajustado o con variables no relevantes para la predicción. Lo ideal es usar el coeficiente de determinación ajustado, ya que considera un trade off entre la mejora y la cantidad de variables agregadas, en función de la cantidad de registros del set de datos.

![texto alternativo](https://i.stack.imgur.com/fLrDw.png)

In [ ]:
# Calculo el coeficiente de determinación R2
r2 = r2_score(y_test, y_pred)

# Calculo el R2 ajustado
n = X_test.shape[0]
p = len(X_train.columns)
r2_ajustado = 1 - ( (1-r2) * ( (n-1) / ( n-p-1 ) ) )

# Muestro los coeficientes
print('Coeficiente de determinación R2: %.4f'% r2)
print('Coeficiente de determinación ajustado: %.4f'% r2_ajustado)

#### Quitamos variables no relevantes
Veamos como cambia el modelo si quitamos las variables no interesantes al objetivo de predicción.


In [ ]:
var_ok = ['MODELO_ATENCION_RENTA_MASIVA','MODELO_ATENCION_JOVENES','MODELO_ATENCION_PREMIUM',
          'MODELO_ATENCION_EMPRESAS','Desc_Resp_Cerrada','VISITAS_CAJA','MODELO_ATENCION_NORMAL']

# Quitamos las columnas que no interesan
X_train2 = X_train[var_ok]
X_test2 = X_test[var_ok]

# Entrenamos y realizamos la predicción sobre el set de testing
lr = LinearRegression()
lr = lr.fit(X_train2, y_train)
y_pred = lr.predict(X_test2)

# MSE - Error cuadratico medio
print('MSE: %.5f'% mean_squared_error(y_test, y_pred))
print('RMSE: %.5f'% math.sqrt(mean_squared_error(y_test, y_pred)))

# Calculo el coeficiente de determinación R2
r2 = r2_score(y_test, y_pred)

# Calculo el R2 ajustado
n = X_test2.shape[0]
p = len(X_test2.columns)
r2_ajustado = 1 - ( (1-r2) * ( (n-1) / ( n-p-1 ) ) )

# Muestro los coeficientes
print('Coeficiente de determinación R2: %.4f'% r2)
print('Coeficiente de determinación ajustado: %.4f'% r2_ajustado)

#### Distribución de los errores
Vemos que la distribución de los errores mejora.

In [ ]:
errores = [diff for diff in y_test - (y_pred)]

media_errores = np.mean(errores)
mediana_errores = np.median(errores)

print("Media de los errores: "+str(media_errores))
print("Mediana de los errores: "+str(mediana_errores))

plt.figure(figsize=(20,10))
sns.distplot(errores, hist=True, kde=True,
             bins=100, color = 'darkblue',
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})

## 4. Construcción del modelo usando GridSearchCV con optimización de hiperparámetros

Las clase **GridSearchCV** se utiliza para automatizar la selección de los parámetros de un modelo, aplicando para ello la técnica de validación cruzada. Partiendo de un modelo y un conjunto de parámetros, GridSearchCV prueba múltiples combinaciones y selecciona los valores de los parámetros que ofrecen mayor rendimiento para un modelo y conjunto de datos.

Los parámetros a optimizar son:

Medida            | Que hace
------------------|-------------
alpha         | Indica el nivel de ajuste a realizar, cuanto mayor menos overfitting
fit_intercept      | Indica si se quiere agregar o no intercepto
max_iter    | Maxima cantidad de iteraciones para converger
tol    | Nivel de tolerancia
normalize  | Indica si es necesario normalizar los datos
solver | Tipo de algoritmo interno a aplicar

Cada uno puede definir sus propios rangos de valores para cada parámetro, considerando que cuantos más parámetros se usen, el tiempo de procesamiento crece exponencialmente.

### 4.a) Definición de parámetros

In [ ]:
model = Ridge()
model.get_params().keys()

In [ ]:
# Definimos los parametros a evaluar:

PARAMETROS = {"fit_intercept":[True, False],
              "tol":          [0.00001, 0.00005, 0.0001, 0.0005],
              "max_iter":     [100, 200, 1000],
              "alpha":        [0.001, 0.01, 0.05, 0.1, 1, 10]}

k_n_jobs = 1 # numero de jobs a ejecutar en paralelo

# Hacemos la búsqueda con GridSearchCV
model = Ridge()
#model = Lasso()
gs = GridSearchCV(model,
                  PARAMETROS,
                  n_jobs=k_n_jobs,
                  refit=True,
                  scoring='neg_root_mean_squared_error', #Definimos la metrica RMSE Raiz del Error Cuadrático Medio
                  cv=KFold(n_splits=10,shuffle=True,random_state=1), #Cross Validation de 5 capas
                  verbose=1)
gs.fit(X_train2, y_train)

# Mostramos los mejores resultados obtenidos
print(gs.best_estimator_)

In [ ]:
# Uso el mejor modelo para predecir
y_pred = gs.best_estimator_.predict(X_test2)

# MSE - Error cuadratico medio
print('Puntaje del modelo en CV: {:.5f}'.format(gs.best_score_))
print('Puntaje del modelo en TESTING: {:.5f}'.format(math.sqrt(mean_squared_error(y_test, y_pred))))

# Calculo el coeficiente de determinación R2
r2 = r2_score(y_test, y_pred)

# Calculo el R2 ajustado
n = X_test2.shape[0]
p = len(X_test2.columns)
r2_ajustado = 1 - ( (1-r2) * ( (n-1) / ( n-p-1 ) ) )

# Muestro los coeficientes
print('Coeficiente de determinación R2: %.4f'% r2)
print('Coeficiente de determinación ajustado: %.4f'% r2_ajustado)

Para ver el resultado final, reentrenamos al modelo y mostramos en un dataframe la comparación entre los valores reales, los predichos y su diferencia

In [ ]:
val_real = pd.Series(y_test.values)
val_pred = pd.Series(y_pred)

predicciones = pd.concat([val_real.rename('Valor real'),val_pred.rename('Valor Pred') ,abs(val_real-val_pred).rename('Dif(+/-)')] ,  axis=1)
predicciones = predicciones.sort_values(by='Dif(+/-)')
predicciones.head(10)

In [ ]:
predicciones.tail(20)

### 4.b) Podemos definir una métrica propia para considerar Acierto o Error
Por ejemplo, consideramos error las predicciones donde el error es superior a X%

In [ ]:
predicciones.columns

In [ ]:
predicciones['ERROR_PORC'] = predicciones['Dif(+/-)'] / predicciones['Valor real']
print("Errores: "+str(predicciones[predicciones['ERROR_PORC'] > 0.7].shape[0] / predicciones.shape[0]))
print("Total: "+str(predicciones.shape[0]))

##**Tips**:
### Versión antigua de Regresion Lineal
Para conocer la relevancia o no de las variables (en función del p-value) se puede usar la libreria original de stadísticas en python **statsmodels**:

    import statsmodels.api as sm
    from scipy import stats

    X2 = sm.add_constant(X_train)
    est = sm.OLS(y_train, X2)
    est2 = est.fit()

    print(est2.summary())

### Uso de Google Colab
Si se desea usar el poder de computo de Google Colab sin necesidad de estar sentado en la notebook/pc, se pueden realizar los siguientes pasos:
- Poner a ejecutar todas las celdas: "Entorno de ejecución" -> "Ejecutar todas"
- Activar la consola de Google Chrome presionando F12.
- Ir a la pestaña Console.
- Ejecutar el siguiente código javascript:

      function ConnectButton(){
        console.log("Connect pushed");
        document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect-icon").click()
      }
      setInterval(ConnectButton,60000);
- Esto va a presionar continuamente el botón conectar del Colab y va a mantener la sesión activa.